# Text preprocessing for strategy score

In [ ]:
import pandas as pd
import numpy as np
import pathlib
import re
import concurrent.futures

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
!pip install gcld3

     |████████████████████████████████| 3.8MB 5.6MB/s 


In [ ]:
import gcld3
detector = gcld3.NNetLanguageIdentifier(min_num_bytes=0, max_num_bytes=10000)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
MAX_THREADS = 30

In [ ]:
df_startups = pd.read_stata("drive/My Drive/Capstone Shared Docs/data/all_deals.dta")
df_startups_unique = df_startups.drop_duplicates(["portfoliocompanyid"],keep="first")

df_public = pd.read_stata("drive/My Drive/Capstone Shared Docs/data/all_public_firms.dta")
df_public_unique = df_public.drop_duplicates(["ÿþmark"],keep="first")

In [ ]:
df_startups_unique.head()

,portfoliocompanyid,website,dealid,portfoliocompany,background,investorsbuyersfirms,portfoliocompanycountry,dealdescription,stage,dealdate,dealcurrency,dealsizecurrmn,dealsizeusdmn,portfoliocompanystatus,dealstatus,investorcountry,investorregion,investorcity,investorstatecounty,portfoliocompanyregion,portfoliocompanycity,portfoliocompanystatecounty,portfoliocompanywebsite,companyrevenuecurrmn,entryrevenuemultiple,ebitdacurrmn,entryebitdamultiple,investorsbuyersfunds,industryclassification,primaryindustry,subindustries,industryverticals,boughtfromsellersfirms,leadpartners,boardrepresentatives,dealsizeeurmn,acquiredshare,financialadvisorsbuyers,financialadvisorssellers,legaladvisorsbuyers,legaladvisorssellers,has_no_background,incyear,_merge,has_website
0,26178,www.barracuda.com,V114584,"Barracuda Networks, Inc.","Founded in 2002 and based in California, Unite...",Focus Ventures,US,"In September 2006, Barracuda Networks, Inc. ra...",Series D/Round 4,2006-09-26,,NaN,NaN,Realised,Completed,US,North America,Los Altos,CA,North America,Campbell,CA,www.barracuda.com,NaN,NaN,NaN,NaN,Focus Ventures II,Information Technology,IT Security/Cybersecurity,"Storage, IT Security/Cybersecurity, Monitoring...",Cloud Computing,,,,NaN,NaN,,,,,0,2002,matched (3),1
1,26580,www.fleetcor.com,V18681,"FleetCor Technologies, Inc.","Founded in 2000 and based in Georgia, US, Flee...",Bain Capital,US,"In June 2005, FleetCor Technologies raised USD...",Series D/Round 4,2005-06-01,USD,75.0,75.0,Realised,Completed,US,North America,Boston,MA,North America,Atlanta,GA,www.fleetcor.com,NaN,NaN,NaN,NaN,,Information Technology,Software,"e-Financial, Accounting/Finance Software",Processing & Payment Infrastructure,,Andrew Balson,,67.33,NaN,,,,,0,2000,matched (3),1
2,26635,www.leafgroup.com,V4520,Leaf Group Ltd.,"Founded in 2006 and based in California, US, L...","Oak Investment Partners, Generation Partners, ...",US,"In March 2008, returning investors 3i, Generat...",Series D/Round 4,2008-03-24,USD,35.0,35.0,Active,Completed,"US, UK","North America, Europe","Greenwich, Boston, New York, London","CT, MA, NY",North America,Santa Monica,CA,www.leafgroup.com,NaN,NaN,NaN,NaN,"Spectrum Equity Investors V, Oak Investment Pa...",Information Technology,Internet,"Advertising Agencies & Media Buyers, eMarketin...","AdTech, Cloud Computing",,John Hawkins,,31.42,NaN,,,,,0,2006,matched (3),1
3,26957,www.sawayasegalas.com,V9012,Sawaya Segalas & Co.,"Founded in 2001 and based in New York, United ...","VOSS of Norway ASA, Centra Capital, Juggernaut...",US,"In August 2011, Sawaya Segalas & Co. raised US...",Growth Capital/Expansion,2011-08-03,USD,18.0,18.0,Active,Completed,US,North America,"New York, Pennsauken, Washington","NY, NJ, DC",North America,New York,NY,www.sawayasegalas.com,NaN,NaN,NaN,NaN,,Financial & Insurance Services,Financial Services,Investment Banking,,,,,16.16,NaN,,,,,0,2001,matched (3),1
4,28167,www.activeinterestmedia.com,V213946,Active Interest Media Inc.,"Founded in 2003 and based in Colorado, US, Act...","Pocket Outdoor Media, Inc., Zone 5 Ventures",US,"In June 2020, Pocket Outdoor Media, Inc., a po...",Add-on,2020-06-30,,NaN,NaN,Active,Completed,US,North America,"Boulder, San Francisco","CO, CA",North America,Boulder,CO,www.activeinterestmedia.com,NaN,NaN,NaN,NaN,,Business Services,Marketing/Advertising,,,,,,NaN,NaN,,,,,0,2003,matched (3),1


In [ ]:
drive_folder= "drive/My Drive/Capstone Shared Docs/result"
txt_folder = drive_folder + "/Txt Files Public and Startups"
txt_folder_2 = txt_folder.replace(" ","\ ")
!ls $txt_folder_2

'Public Companies'   Startups


In [ ]:
MAX_WORDS = 1000000

In [ ]:
import math

In [ ]:
def get_text_paths(p,t):
  lst =  list(p.glob('*.txt'))
  return lst

def read_text_file(p):
  n = p.name.split("_")[0]
  file = open(p)

  data = file.read()
  words = data.split()
  if len(words)> MAX_WORDS:
    return n,""

  # Remove first two and last characters, because of ascii encoding
  text = data.replace("***///***","\n")[2:-1]
  # remove ""
  text_list = text.split(" ")
  text_list = [x for x in text_list if x!=""]
  text = " ".join(text_list)
  file.close()
  return n,text

def get_text(files_list):
  threads = min(MAX_THREADS, len(files_list))
  company_text = {}
  data_futures = []
  with concurrent.futures.ThreadPoolExecutor(max_workers = threads) as thread_pool_executor:
    data_futures = [thread_pool_executor.submit(read_text_file, p) for p in files_list]
  for data_future in concurrent.futures.as_completed(data_futures):
      future = data_future.result()
      if future[1]!="":
        company_text[future[0]] = future[1]
  return company_text
    

def create_data_frame(company_text, company_type):
  company_frame = pd.DataFrame.from_dict(company_text, orient='index')
  company_frame.columns = ['text']
  # company_frame['companyid'] = company_frame.index
  # company_frame.index = np.arange(0,len(company_frame.companyid))
  company_frame = company_frame.assign(comp_type=company_type)
  stop_words_l=stopwords.words('english')
  company_frame['text_cleaned']=company_frame.text.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l) )
  company_frame['text_cleaned_with_sw']=company_frame.text.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() ))
  company_frame["compid"] = company_frame.index
  company_frame.reset_index(inplace=True,drop=True)

  return company_frame

def get_path(c_type,year):
  if c_type=="S":
    folder_name = txt_folder + f"/Startups/Startup {year}"
  else:
    folder_name = txt_folder + f"/Public Companies/Public {year}"
  return pathlib.Path(folder_name)


def build_data_frame(c_type,year):
  path = get_path(c_type, year)
  files_path = get_text_paths(path,c_type)
  print(f"{c_type} {year} text files: "+str(len(files_path)))
  files = get_text(files_path)
  df = create_data_frame(files,c_type)
  del files_path
  del files
  return df

def merge_and_resolve_language(df1,df2):
  all_f = pd.concat([df1, df2]).reset_index(drop=True)
  del df1
  del df2
  # all_f.drop_duplicates(subset="text", keep="first",inplace=True)

  lan = []
  rel_lan = []
  for i,r in all_f.iterrows():
    t =r.text_cleaned
    t = str(t)
    
    result = detector.FindLanguage(text=t)
    lan.append(result.language)
    rel_lan.append(result.is_reliable)
  all_f['language'] = lan
  all_f['Rela_language'] = rel_lan

  # all_f_en = all_f[(all_f.language=='en') ].copy() #| ( (all_f.language!='en') & (all_f.language==False) )
  all_f_en = all_f
  del all_f
  # all_f_en.reset_index(drop=True,inplace=True)

  all_f_en["text_len"] = all_f_en.text_cleaned.apply(lambda x: len(x))
  all_f_en["words_len"] = all_f_en.text_cleaned.apply(lambda x: len(x.split()))

  return all_f_en

In [ ]:
def add_informative_columns(all_f, year):
  s_path = pathlib.Path(txt_folder +f"/Startups/Startup {year}")
  p_path = pathlib.Path(txt_folder +f"/Public Companies/Public {year}")

  startup_code_to_name = {}
  startup_code_to_website = {}
  public_code_to_name = {}
  public_code_to_website = {}

  for p in get_text_paths(s_path,"S"):
    n = int(p.name.split("_")[0])
    startup_code_to_name[n] = df_startups_unique[df_startups_unique.portfoliocompanyid==n].iloc[0].portfoliocompany
    startup_code_to_website[n] = df_startups_unique[df_startups_unique.portfoliocompanyid==n].iloc[0].website

  for p in get_text_paths(p_path,"P"):
    n = str(p.name.split("_")[0])
    public_code_to_name[n] = df_public_unique[df_public_unique["ÿþmark"]==n].iloc[0].companyname
    public_code_to_website[n] = df_public_unique[df_public_unique["ÿþmark"]==n].iloc[0].websiteaddress

  startup_code_to_info = df_startups_unique.set_index("portfoliocompanyid").background.to_dict()
  public_code_to_info = df_public_unique.set_index("ÿþmark").descriptionandhistory.to_dict()

  all_f.loc[all_f.comp_type=="S","website"] = all_f[all_f.comp_type=="S"].compid.astype(int).map(startup_code_to_website)
  all_f.loc[all_f.comp_type=="S","name"] = all_f[all_f.comp_type=="S"].compid.astype(int).map(startup_code_to_name)
  all_f.loc[all_f.comp_type=="S","info"] = all_f[all_f.comp_type=="S"].compid.astype(int).map(startup_code_to_info)

  all_f.loc[all_f.comp_type=="P","website"] = all_f[all_f.comp_type=="P"].compid.astype(str).map(public_code_to_website)
  all_f.loc[all_f.comp_type=="P","name"] = all_f[all_f.comp_type=="P"].compid.astype(str).map(public_code_to_name)
  all_f.loc[all_f.comp_type=="P","info"] = all_f[all_f.comp_type=="P"].compid.astype(str).map(public_code_to_info)

In [ ]:
 def preprocess_year(year):
  df_1 = build_data_frame("S",year)
  df_2 = build_data_frame("P",year)
  # df_1 = pd.DataFrame(columns = df_2.columns)
  df_3 = merge_and_resolve_language(df_1,df_2)
  del df_1
  del df_2
  add_informative_columns(df_3,year)
  return df_3

In [ ]:
# Run this to create csv for all years
for i in range(2010,2011):
  data = preprocess_year(i)
  data.to_csv(drive_folder+f"/complete_df/year_{i}.csv",index=False)
  del data
  print(f"Year {i} completed!")

S 2010 text files: 430
P 2010 text files: 9070
Year 2010 completed!


In [ ]:
import pandas as pd
drive_folder= "drive/My Drive/Capstone Shared Docs/result"

In [ ]:
df_folder = drive_folder+"/complete_df"
df_folder2 = df_folder.replace(" ","\ ")

In [ ]:
!ls $df_folder2

year_2000.csv  year_2005.csv  year_2010.csv  year_2015.csv     year_2019.csv
year_2001.csv  year_2006.csv  year_2011.csv  year_2016_p1.csv
year_2002.csv  year_2007.csv  year_2012.csv  year_2016_p2.csv
year_2003.csv  year_2008.csv  year_2013.csv  year_2017.csv
year_2004.csv  year_2009.csv  year_2014.csv  year_2018.csv


In [ ]:
!cat drive/My\ Drive/Capstone\ Shared\ Docs/result/complete_df/year_2016*.csv > drive/My\ Drive/Capstone\ Shared\ Docs/result/complete_df/year_2016.csv

In [ ]:
!ls drive/My\ Drive/Capstone\ Shared\ Docs/result/complete_df

year_2000.csv  year_2005.csv  year_2010.csv  year_2015.csv     year_2019.csv
year_2001.csv  year_2006.csv  year_2011.csv  year_2016_p1.csv
year_2002.csv  year_2007.csv  year_2012.csv  year_2016_p2.csv
year_2003.csv  year_2008.csv  year_2013.csv  year_2017.csv
year_2004.csv  year_2009.csv  year_2014.csv  year_2018.csv


In [ ]:
df_folder = drive_folder+"/complete_df"

In [ ]:
csv_path = df_folder+"/year_2014.csv"
csv_path2 = csv_path.replace(" ","\ ")

In [ ]:
df = pd.read_csv(csv_path,usecols=["comp_type","text_cleaned","compid","language","text_len","words_len","website","name","info"])
df.shape

(11351, 9)

In [ ]:
!head $csv_path2 -n3 

text,comp_type,text_cleaned,text_cleaned_with_sw,compid,language,Rela_language,text_len,words_len,website,name,info
"Future home of something quite cool. If you're the site owner , log in to launch this site If you are a visitor , check back soon.",S,future home something quite cool  you re site owner   log launch site visitor   check back soon ,future home of something quite cool  if you re the site owner   log in to launch this site if you are a visitor   check back soon ,225215,en,True,96,16,www.courtbuddy.com,"The Court Buddy Company, Inc.","Established in 2014 and based in Florida, US, The Court Buddy Company, Inc. operates an online platform that allows users to choose legal services on an a-la-carte basis."
"Get a Soofa Soofa Installations Team Careers Blog Get a Soofa Soofa Installations Team Careers Blog Much more than a bench Soofa Much more than a bench SCROLL DOWN Much more than a bench Soofa MUCH MORE THAN A BENCH A solar-powered hub for smart, social and sustainable space

In [ ]:
df.head()

,comp_type,text_cleaned,compid,language,text_len,words_len,website,name,info
0,S,future home something quite cool you re site ...,225215,en,96,16,www.courtbuddy.com,"The Court Buddy Company, Inc.","Established in 2014 and based in Florida, US, ..."
1,S,get soofa soofa installations team careers blo...,161770,en,12396,1655,www.soofa.co,"Changing Environments, Inc.","Founded in 2014 and based in Massachusetts, US..."
2,S,menu product contact login sign close product ...,245173,en,17443,2113,www.liveoak.net,"LiveOak Technologies, Inc.","Founded in 2014 and based in Texas, US, LiveOa..."
3,S,learn see coaches blog sign log messages accou...,167606,en,135481,17169,www.coachmarket.com,Meister Internet Group LLC,"Founded in 2014 and based in New York, US, Mei..."
4,S,products services insight duo tascent mobile...,175875,en,11803,1452,www.tascent.com,"Tascent, Inc.","Founded in 2014 and based in California, US, T..."


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
drive_folder= "drive/My Drive/Capstone Shared Docs/result"

In [ ]:
df = pd.read_csv(drive_folder + "/complete_df/year_2000.csv")
df.head()

,text,comp_type,text_cleaned,text_cleaned_with_sw,compid,language,Rela_language,text_len,words_len,website,name,info
0,You See! We even put this handy Search Script ...,S,see even put handy search script make site so...,you see we even put this handy search script ...,334249,en,True,412,70,www.glitch.com,"Glitch, Inc.","Founded in 2000 and based in New York, US, Gli..."
1,"""Next-generation Storage Networking Infrastruc...",S,next generation storage networking infrastruc...,next generation storage networking infrastruc...,99974,en,True,171,22,www.neoscale.com,NeoScale Systems Inc.,"Founded in 2000, NeoScale Systems, Inc. provid..."
2,"Aloha Friends~ ""Kalepa"" means ""Merchant""; Pron...",S,aloha friends kalepa means merchant pron...,aloha friends kalepa means merchant pron...,264488,en,True,451,56,www.kalepa.com,"Kalepa Networks, Inc.","Founded in 2000 and based in California, US, K..."
3,Company Information Products Services We have ...,S,company information products services eye data...,company information products services we have ...,46517,en,True,9341,1157,www.netmining.com,Netmining,"Established in 2000, based in New York, Netmin..."
4,ll Conducting Soon Copyright 2001 Orchestria C...,S,conducting soon copyright orchestria corp...,ll conducting soon copyright orchestria c...,84585,en,True,91,10,www.orchestria.com,Orchestria,"Founded in 2000, Orchestria provides informati..."


In [ ]:
df.drop_duplicates("website",keep='first',inplace=True)
df[(df.language=="en") & (df.words_len>=25)].comp_type.value_counts()

P    144
S     38
Name: comp_type, dtype: int64

In [ ]:
# for i in range(int(df.year.max())+1,2020):
df = None
for i in range(2000,2020):
  if i in [2016,2019]:
    f = f"/complete_df/year_{i}_short.csv"
  else:
    f = f"/complete_df/year_{i}.csv"
  print(f)
  d = pd.read_csv(drive_folder+f,usecols=["website","comp_type","language"])
  d.drop_duplicates("website",keep="first",inplace=True)

  n = d.shape[0]
  comp_type = d.comp_type.value_counts()
  s = comp_type["S"]
  p = comp_type["P"]
  l = d.language.value_counts()
  en = l["en"]
  other_lan = n - en
  if i==2000:
    df = pd.DataFrame({"year":[],"total":[],"startups":[],"public":[],"english":[],"other_lang":[]})

  df = df.append({"year":i,"total":n,"startups":s,"public":p,"english":en,"other_lang":other_lan},ignore_index=True)
  df = df.astype(int)
  df.to_csv(drive_folder + "/counts_per_year.csv",index=False)
  print(f"Year {i} done!")
df

/complete_df/year_2000.csv
Year 2000 done!
/complete_df/year_2001.csv
Year 2001 done!
/complete_df/year_2002.csv
Year 2002 done!
/complete_df/year_2003.csv
Year 2003 done!
/complete_df/year_2004.csv
Year 2004 done!
/complete_df/year_2005.csv
Year 2005 done!
/complete_df/year_2006.csv
Year 2006 done!
/complete_df/year_2007.csv
Year 2007 done!
/complete_df/year_2008.csv
Year 2008 done!
/complete_df/year_2009.csv
Year 2009 done!
/complete_df/year_2010.csv
Year 2010 done!
/complete_df/year_2011.csv
Year 2011 done!
/complete_df/year_2012.csv
Year 2012 done!
/complete_df/year_2013.csv
Year 2013 done!
/complete_df/year_2014.csv
Year 2014 done!
/complete_df/year_2015.csv
Year 2015 done!
/complete_df/year_2016_short.csv
Year 2016 done!
/complete_df/year_2017.csv
Year 2017 done!
/complete_df/year_2018.csv
Year 2018 done!
/complete_df/year_2019_short.csv
Year 2019 done!


,year,total,startups,public,english,other_lang
0,2000,221,49,172,205,16
1,2001,3975,51,3924,3755,220
2,2002,4055,44,4011,3865,190
3,2003,4656,104,4552,4404,252
4,2004,4655,142,4513,4457,198
5,2005,3680,147,3533,3562,118
6,2006,5357,181,5176,5202,155
7,2007,5656,221,5435,5475,181
8,2008,5237,211,5026,5068,169
9,2009,5390,294,5096,5215,175


In [ ]:
drive_folder= "drive/My Drive/Capstone Shared Docs/strategy score"

In [ ]:
df = pd.DataFrame({"year":[],"total":[],"startups":[],"public":[],"english":[],"other_lang":[]})
for i in range(2000,2020):
  f = f"/tfidf_{i}.csv"
  print(f)
  d = pd.read_csv(drive_folder+f)
  d["score_year"] = i
  # d.set_index("compid",inplace=True)
  dic = df_startups_unique.groupby("portfoliocompanyid").incyear.first().to_dict()
  d["incyear"] = d.compid.astype(int).map(dic)
  break

  n = d.shape[0]
  comp_type = d.comp_type.value_counts()
  s = comp_type["S"]
  p = comp_type["P"]
  l = d.language.value_counts()
  en = l["en"]
  other_lan = n - en
  if i==2000:
    df = pd.DataFrame({"year":[],"total":[],"startups":[],"public":[],"english":[],"other_lang":[]})

  df = df.append({"year":i,"total":n,"startups":s,"public":p,"english":en,"other_lang":other_lan},ignore_index=True)
  df = df.astype(int)
  df.to_csv(drive_folder + "/counts_per_year.csv",index=False)
  print(f"Year {i} done!")
d

/tfidf_2000.csv


,text,comp_type,text_cleaned,text_cleaned_with_sw,compid,language,Rela_language,text_len,words_len,website,name,info,Dict_similarity,Strat_score,score_year,incyear
0,You See! We even put this handy Search Script ...,S,see even put handy search script make site so...,you see we even put this handy search script ...,334249,en,True,412,70,www.glitch.com,"Glitch, Inc.","Founded in 2000 and based in New York, US, Gli...","{104274: 0.046667066894957254, 264488: 0.04518...",0.960860,2000,2000
1,"Aloha Friends~ ""Kalepa"" means ""Merchant""; Pron...",S,aloha friends kalepa means merchant pron...,aloha friends kalepa means merchant pron...,264488,en,True,451,56,www.kalepa.com,"Kalepa Networks, Inc.","Founded in 2000 and based in California, US, K...","{334249: 0.045180478111755996, 76058: 0.037721...",0.966607,2000,2000
2,Company Information Products Services We have ...,S,company information products services eye data...,company information products services we have ...,46517,en,True,9341,1157,www.netmining.com,Netmining,"Established in 2000, based in New York, Netmin...","{157752: 0.11677840721692541, 8020: 0.09825727...",0.907142,2000,2000
3,"nano (nah-no), noun, 1. Technology company tha...",S,nano nah no noun technology company eng...,nano nah no noun technology company tha...,30078,en,True,353,42,www.nano.com,"Nanomix, Inc.","Founded in 2000 and based in California, US, N...","{9541: 0.0568081005229298, 2720: 0.05680810052...",0.943256,2000,2000
4,Welcome to Starent Networks Corporation Introd...,S,welcome starent networks corporation introduct...,welcome to starent networks corporation introd...,76058,en,True,400,39,www.starentnetworks.com,Starent Networks Corporation,"Founded in 2000, Starent Networks, Corp. is a ...","{2169: 0.2116046958318176, 274498: 0.139690901...",0.876258,2000,2000
5,The Company Shared Spectrum Company Shared Spe...,S,company shared spectrum company shared spectru...,the company shared spectrum company shared spe...,121009,en,True,436,50,www.sharedspectrum.com,Shared Spectrum Company,"Founded in 2000, based in US, Shared Spectrum ...","{99970: 0.08461290406919861, 9541: 0.049810327...",0.943229,2000,2000
6,Fishbowl provides restaurant companies with a ...,S,fishbowl provides restaurant companies simple ...,fishbowl provides restaurant companies with a ...,93048,en,True,737,93,www.fishbowl.com,Fishbowl Inc.,"Founded in 2000 and based in Virginia, US, Fis...","{336164: 0.05664432264450055, 9541: 0.04413361...",0.953364,2000,2000
7,More than 90% of power quality disruptions are...,S,power quality disruptions caused voltage s...,more than of power quality disruptions are...,238725,en,True,1219,165,www.powerstandards.com,Power Standards Lab Inc.,"Established in 2000 and based in California, U...","{6131: 0.0601605911919491, 942: 0.052340896951...",0.958351,2000,2000
8,MAKE AN OFFER ON THIS DOMAIN!:$ Find What Your...,S,make offer domain find looking services co...,make an offer on this domain find what your...,70645,en,True,709,95,www.operative.com,"Operative Media, Inc.","Established in 2000 and based in New York, US,...","{227621: 0.33252062443694547, 1458: 0.21930980...",0.849880,2000,2000
9,How can you manage what you can't define? How ...,S,manage can t define secure can t manage lume...,how can you manage what you can t define how ...,82560,en,True,1036,122,www.lumeta.com,Lumeta Corporation,"Established in 2000 and based in New Jersey, U...","{227621: 0.11118045152993916, 274498: 0.052671...",0.937366,2000,2000
